# Estimating Joint and Non-Mandatory Tour Destination Choice

This notebook illustrates how to re-estimate both the joint tour destination and 
non-mandatory tour destination choice models.  These models share parameters and need
to be re-estimated together. This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelnames = ("non_mandatory_tour_destination", "joint_tour_destination")

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelnames,
    edb_directory="output-est-mode/estimation_data_bundle/{name}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_destination/non_mandatory_tour_destination_coefficients.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_destination/non_mandatory_tour_destination_SPEC.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_destination/non_mandatory_tour_destination_alternatives_combined.parquet
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_destination/non_mandatory_tour_destination_choosers_combined.parquet
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_destination/non_mandatory_tour_destination_landuse.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_destination/non_mandatory_tour_destination_size_terms.csv
loading from output-est-mode/estimation_data_bundle/joint_tour_destination/non_mandatory_tour_destination_coefficients.csv
loading from output-est-mode/estimation_data_bundle/joint_tour_destination/joint_tou

The resulting model is actually a larch `ModelGroup`, which exposes a similar 
API as a single model, and allows for joint estimation of all parameters across
all models in the group.

# Review data loaded from EDB

Next we can review what was read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

## coefficients

In [4]:
data[0].coefficients

,value,constrain
coefficient_name,,
coef_mode_logsum,0.6755,F
coef_escort_dist_0_2,-0.1499,F
coef_eatout_dist_0_2,-0.5609,F
coef_eatout_social_0_2,-0.5609,F
coef_othdiscr_dist_0_2,-0.1677,F
coef_escort_dist_2_5,-0.8671,F
coef_shopping_dist_2_5,-0.5655,F
coef_eatout_dist_2_5,-0.3192,F
coef_othmaint_dist_2_5,-0.6055,F


## alt_values

In [5]:
data[0].alt_values

,tour_id,alt_dest,variable_label0000,variable_label0001,variable_label0002,variable_label0003,variable_label0004,variable_label0005,variable_label0006,variable_label0007,variable_label0008
0,1870,8,1.0,1.00,2.06,0.0,0.0,7.492867,False,-0.349936,4.970833
1,1870,16,1.0,1.00,2.74,0.0,0.0,8.289751,False,-0.487843,4.510890
2,1870,17,1.0,1.00,3.00,0.1,0.0,7.651656,False,-0.537413,5.289744
3,1870,24,1.0,1.00,1.77,0.0,0.0,7.111745,False,-0.401546,5.208261
4,1870,26,1.0,1.00,1.78,0.0,0.0,6.132923,False,-0.354362,6.192038
...,...,...,...,...,...,...,...,...,...,...,...
712367,310220296,1428,1.0,1.00,1.02,0.0,0.0,7.202319,False,-0.662897,3.354280
712368,310220296,1430,1.0,1.00,0.90,0.0,0.0,7.344198,False,-0.447759,4.749178
712369,310220296,1436,1.0,1.00,0.05,0.0,0.0,6.819036,False,-0.234562,3.150228
712370,310220296,1437,1.0,0.02,0.00,0.0,0.0,7.407333,False,0.312912,2.531655


In [6]:
data[1].alt_values

,tour_id,alt_dest,variable_label0000,variable_label0001,variable_label0002,variable_label0003,variable_label0004,variable_label0005,variable_label0006,variable_label0007,variable_label0008
0,7853686,5,1.0,1.0,3.0,5.42,0.000000,8.012729,False,-3.683188,5.037499
1,7853686,9,1.0,1.0,3.0,4.65,0.000000,7.841204,False,-3.477110,4.420550
2,7853686,14,1.0,1.0,3.0,5.74,0.000000,6.755056,False,-3.777769,5.641640
3,7853686,19,1.0,1.0,3.0,4.64,0.000000,6.889447,False,-3.279737,5.371069
4,7853686,29,1.0,1.0,3.0,5.56,0.000000,6.470564,False,-3.709382,6.596995
...,...,...,...,...,...,...,...,...,...,...,...
32626,305246245,940,1.0,1.0,3.0,10.00,0.220000,6.621308,False,-0.893075,7.328697
32627,305246245,971,1.0,1.0,3.0,9.43,0.000000,8.112977,False,-0.434306,5.742782
32628,305246245,1059,1.0,1.0,3.0,10.00,5.740000,7.339134,False,-1.306979,7.269407
32629,305246245,1155,1.0,1.0,3.0,10.00,6.820000,5.311707,False,-1.512147,9.425679


## chooser_data

In [7]:
data[0].chooser_data

,tour_id,model_choice,override_choice,person_id,tour_type,home_zone_id
0,1870,70,26,45,othdiscr,52
1,20468,138,161,499,escort,124
2,27055,133,185,659,social,132
3,38877,115,147,948,escort,148
4,38904,107,104,948,social,148
...,...,...,...,...,...,...
29664,310197956,1374,1384,7565803,shopping,1386
29665,310202376,1402,1402,7565911,othdiscr,1402
29666,310202384,1402,1402,7565911,shopping,1402
29667,310212634,1405,1422,7566161,shopping,1421


In [8]:
data[1].chooser_data

,tour_id,model_choice,override_choice,person_id,tour_type,home_zone_id
0,7853686,131,131,191553,eatout,217
1,7902318,321,323,192739,shopping,322
2,7909776,350,320,192921,othdiscr,330
3,7988742,527,637,194847,othdiscr,526
4,8103874,591,687,197655,shopping,683
...,...,...,...,...,...,...
1272,303771147,484,673,7409052,othdiscr,675
1273,304144212,796,843,7418154,social,806
1274,304463637,9,38,7425942,othdiscr,1043
1275,305125582,29,98,7442087,othdiscr,50


## landuse

In [9]:
data[0].landuse

,DISTRICT,SD,county_id,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,AGE0519,...,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index,is_cbd
zone_id,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,46,82,20.3,1.0,15.00000,27318,7,...,0,0.0,0.00000,0.0,3,5.89564,2.875000,1707.375000,2.870167,False
2,1,1,1,134,240,31.1,1.0,24.79297,42078,19,...,0,0.0,0.00000,0.0,1,5.84871,5.195214,1631.374751,5.178722,False
3,1,1,1,267,476,14.7,1.0,2.31799,2445,38,...,0,0.0,0.00000,0.0,1,5.53231,80.470405,736.891913,72.547987,False
4,1,1,1,151,253,19.3,1.0,18.00000,22434,20,...,0,0.0,0.00000,0.0,2,5.64330,7.947368,1180.736842,7.894233,False
5,1,1,1,611,1069,52.7,1.0,15.00000,15662,86,...,0,0.0,72.14684,0.0,1,5.52555,38.187500,978.875000,36.753679,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,34,34,9,2724,6493,1320.0,630.0,69.00000,1046,1013,...,4,0.0,0.00000,0.0,1,1.12116,3.896996,1.496423,1.081235,False
1451,34,34,9,2016,4835,664.0,379.0,43.00000,757,757,...,4,0.0,0.00000,0.0,1,1.17116,4.777251,1.793839,1.304140,False
1452,34,34,9,2178,5055,1068.0,602.0,35.00000,2110,789,...,4,0.0,0.00000,0.0,1,1.17587,3.419152,3.312402,1.682465,False


## spec

In [10]:
data[0].spec

,Label,Expression,Description,escort,escortkids,escortnokids,shopping,eatout,othmaint,social,othdiscr
0,variable_label0000,"@skims['DIST'].clip(0,1)","Distance, piecewise linear from 0 to 1 miles",coef_escort_dist_0_2,coef_escort_dist_0_2,coef_escort_dist_0_2,0,coef_eatout_dist_0_2,0,coef_eatout_dist_0_2,coef_othdiscr_dist_0_2
1,variable_label0001,"@(skims['DIST']-1).clip(0,1)","Distance, piecewise linear from 1 to 2 miles",coef_escort_dist_0_2,coef_escort_dist_0_2,coef_escort_dist_0_2,0,coef_eatout_dist_0_2,0,coef_eatout_dist_0_2,coef_othdiscr_dist_0_2
2,variable_label0002,"@(skims['DIST']-2).clip(0,3)","Distance, piecewise linear from 2 to 5 miles",coef_escort_dist_2_5,coef_escort_dist_2_5,coef_escort_dist_2_5,coef_shopping_dist_2_5,coef_eatout_dist_2_5,coef_othmaint_dist_2_5,coef_social_dist_2_5,coef_othdiscr_dist_2_5
3,variable_label0003,"@(skims['DIST']-5).clip(0,10)","Distance, piecewise linear from 5 to 15 miles",coef_escort_dist_5_plus,coef_escort_dist_5_plus,coef_escort_dist_5_plus,coef_shopping_dist_5_plus,coef_eatout_dist_5_plus,coef_othmaint_dist_5_plus,coef_social_dist_5_plus,coef_othdiscr_dist_5_plus
4,variable_label0004,@(skims['DIST']-15.0).clip(0),"Distance, piecewise linear for 15+ miles",coef_escort_dist_5_plus,coef_escort_dist_5_plus,coef_escort_dist_5_plus,coef_shopping_dist_5_plus,coef_eatout_dist_5_plus,coef_othmaint_dist_5_plus,coef_social_dist_5_plus,coef_othdiscr_dist_5_plus
5,variable_label0006,@df['size_term']==0,No attractions,-999,-999,-999,-999,-999,-999,-999,-999
6,variable_label0007,mode_choice_logsum,Mode choice logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum
7,variable_label0008,"@np.minimum(np.log(df.pick_count/df.prob), 60)",Sample of alternatives correction factor,1,1,1,1,1,1,1,1


## size_spec

In [11]:
data[0].size_spec

,TOTHH,RETEMPN,HEREMPN,OTHEMPN,AGE0519,HSENROLL
segment,,,,,,
escort,0.000,0.225,0.144,0.000,0.465,0.166
shopping,0.000,1.000,0.000,0.000,0.000,0.000
eatout,0.000,0.742,0.258,0.000,0.000,0.000
othmaint,0.000,0.482,0.518,0.000,0.000,0.000
social,0.000,0.522,0.478,0.000,0.000,0.000
othdiscr,0.252,0.212,0.272,0.165,0.000,0.098


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [12]:
model.estimate(maxiter=900)

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.0,1
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1
1,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1
coef_eatout_dist_0_2,-0.959646,-0.959646,-0.560900,-25.000000,25.000000,0.0,0
coef_eatout_dist_2_5,-0.427890,-0.427890,-0.319200,-25.000000,25.000000,0.0,0
coef_eatout_dist_5_plus,-0.182199,-0.182199,-0.123800,-25.000000,25.000000,0.0,0
coef_escort_dist_0_2,-0.290871,-0.290871,-0.149900,-25.000000,25.000000,0.0,0
coef_escort_dist_2_5,-0.959038,-0.959038,-0.867100,-25.000000,25.000000,0.0,0
coef_escort_dist_5_plus,-0.264682,-0.264682,-0.213700,-25.000000,25.000000,0.0,0


┣                x: -999                        -999.000000
┃                   0                              0.000000
┃                   1                              1.000000
┃                   coef_eatout_dist_0_2          -0.959646
┃                   coef_eatout_dist_2_5          -0.427890
┃                   coef_eatout_dist_5_plus       -0.182199
┃                   coef_escort_dist_0_2          -0.290871
┃                   coef_escort_dist_2_5          -0.959038
┃                   coef_escort_dist_5_plus       -0.264682
┃                   coef_mode_logsum               0.165346
┃                   coef_othdiscr_dist_0_2        -0.424399
┃                   coef_othdiscr_dist_2_5        -0.615669
┃                   coef_othdiscr_dist_5_plus     -0.171387
┃                   coef_othmaint_dist_2_5        -0.796780
┃                   coef_othmaint_dist_5_plus     -0.156745
┃                   coef_shopping_dist_2_5        -0.704553
┃                   coef_shopping_dist_5_plus     -0.234821
┃                   coef_social_dist_2_5          -0.464176
┃                   coef_social_dist_5_plus       -0.176446
┃                   eatout_HEREMPN                -1.601448
┃                   eatout_RETEMPN                -0.298406
┃                   escort_AGE0519                -0.765963
┃                   escort_HEREMPN                -1.996310
┃                   escort_HSENROLL               -1.872611
┃                   escort_RETEMPN                -1.491655
┃                   othdiscr_HEREMPN              -1.342128
┃                   othdiscr_HSENROLL             -2.287886
┃                   othdiscr_OTHEMPN              -1.905661
┃                   othdiscr_RETEMPN              -1.550175
┃                   othdiscr_TOTHH                -1.378326
┃                   othmaint_HEREMPN              -0.689025
┃                   othmaint_RETEMPN              -0.729811
┃                   shopping_RETEMPN               0.000000
┃                   social_HEREMPN                -0.831065
┃                   social_RETEMPN                -0.650088
┃                   dtype: float64
┣          logloss: 2.770862804895239
┣        d_logloss: -999                         0.000000
┃                   0                            0.000000
┃                   1                            0.000000
┃                   coef_eatout_dist_0_2         0.000068
┃                   coef_eatout_dist_2_5         0.000060
┃                   coef_eatout_dist_5_plus      0.000251
┃                   coef_escort_dist_0_2         0.000236
┃                   coef_escort_dist_2_5        -0.000043
┃                   coef_escort_dist_5_plus      0.000007
┃                   coef_mode_logsum            -0.000027
┃                   coef_othdiscr_dist_0_2      -0.000193
┃                   coef_othdiscr_dist_2_5      -0.000067
┃                   coef_othdiscr_dist_5_plus    0.000015
┃                   coef_othmaint_dist_2_5      -0.000110
┃                   coef_othmaint_dist_5_plus   -0.000227
┃                   coef_shopping_dist_2_5       0.000147
┃                   coef_shopping_dist_5_plus    0.000482
┃                   coef_social_dist_2_5        -0.000027
┃                   coef_social_dist_5_plus     -0.000062
┃                   eatout_HEREMPN               0.000078
┃                   eatout_RETEMPN               0.000000
┃                   escort_AGE0519              -0.000137
┃                   escort_HEREMPN              -0.000064
┃                   escort_HSENROLL             -0.000067
┃                   escort_RETEMPN               0.000000
┃                   othdiscr_HEREMPN             0.000065
┃                   othdiscr_HSENROLL            0.000034
┃                   othdiscr_OTHEMPN            -0.000049
┃                   othdiscr_RETEMPN             0.000009
┃                   othdiscr_TOTHH               0.000000
┃                   othmaint_HEREMPN            -0.000002
┃                   othmaint_

### Estimated coefficients

In [13]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
Parameter,,,,,
-999,-999.,0.00,NA,,0.00
0,0.00,0.00,NA,,0.00
1,1.00,0.00,NA,,0.00
coef_eatout_dist_0_2,-0.960,0.0437,-21.97,***,0.00
coef_eatout_dist_2_5,-0.428,0.0207,-20.70,***,0.00
coef_eatout_dist_5_plus,-0.182,0.00484,-37.68,***,0.00
coef_escort_dist_0_2,-0.291,0.0329,-8.85,***,0.00
coef_escort_dist_2_5,-0.959,0.0161,-59.53,***,0.00
coef_escort_dist_5_plus,-0.265,0.00711,-37.23,***,0.00


# Output Estimation Results

In [14]:
from activitysim.estimation.larch import update_coefficients, update_size_spec
result_dir = data[0].edb_directory/"estimated"

## Write updated utility coefficients

The revised coefficients are written out as one file.  For the MTC
example model, these coefficients are written into the non-mandatory
tour model, and are re-used by the joint tour model.

In [15]:
modelname = modelnames[0]

In [16]:
update_coefficients(
    model, data[0], result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

## Write updated size coefficients

In [17]:
update_size_spec(
    model, data[0], result_dir, 
    output_file=f"{modelname}_size_terms.csv",
)

,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,work_low,workplace,0.000000,0.129129,0.193193,0.383383,0.120120,0.010010,0.164164,0.000000,0.000000,0.000000,0.000000
1,work_med,workplace,0.000000,0.120120,0.197197,0.325325,0.139139,0.008008,0.210210,0.000000,0.000000,0.000000,0.000000
2,work_high,workplace,0.000000,0.110000,0.207000,0.284000,0.154000,0.006000,0.239000,0.000000,0.000000,0.000000,0.000000
3,work_veryhigh,workplace,0.000000,0.093000,0.270000,0.241000,0.146000,0.004000,0.246000,0.000000,0.000000,0.000000,0.000000
4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.592000,0.408000
5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
7,escort,non_mandatory,0.000000,0.229722,0.000000,0.138687,0.000000,0.000000,0.000000,0.474643,0.156948,0.000000,0.000000
8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,eatout,non_mandatory,0.000000,0.786346,0.000000,0.213654,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Write the model estimation report, including coefficient t-statistic and log likelihood

Each model in the ModelGroup can have its own seperate estimation report, as the
common ModelGroup class does not yet implement a common report output.

In [18]:
model[0].to_xlsx(
    result_dir/f"{modelnames[0]}_model_estimation.xlsx", 
    data_statistics=False,
)
model[1].to_xlsx(
    result_dir/f"{modelnames[1]}_model_estimation.xlsx", 
    data_statistics=False,
);

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file and `*_size_terms.csv` file to the configs folder, rename them to `*_coefficients.csv` and `destination_choice_size_terms.csv`, and run ActivitySim in simulation mode.  Note that all the location
and desintation choice models share the same `destination_choice_size_terms.csv` input file, so if you
are updating all these models, you'll need to ensure that updated sections of this file for each model
are joined together correctly.

In [19]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_mode_logsum,0.165346,F
1,coef_escort_dist_0_2,-0.290871,F
2,coef_eatout_dist_0_2,-0.959646,F
3,coef_eatout_social_0_2,-0.560900,F
4,coef_othdiscr_dist_0_2,-0.424399,F
5,coef_escort_dist_2_5,-0.959038,F
6,coef_shopping_dist_2_5,-0.704553,F
7,coef_eatout_dist_2_5,-0.427890,F
8,coef_othmaint_dist_2_5,-0.796780,F
9,coef_social_dist_2_5,-0.464176,F


In [20]:
pd.read_csv(result_dir/f"{modelname}_size_terms.csv")

,index,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,0,work_low,workplace,0.000000,0.129129,0.193193,0.383383,0.120120,0.010010,0.164164,0.000000,0.000000,0.000000,0.000000
1,1,work_med,workplace,0.000000,0.120120,0.197197,0.325325,0.139139,0.008008,0.210210,0.000000,0.000000,0.000000,0.000000
2,2,work_high,workplace,0.000000,0.110000,0.207000,0.284000,0.154000,0.006000,0.239000,0.000000,0.000000,0.000000,0.000000
3,3,work_veryhigh,workplace,0.000000,0.093000,0.270000,0.241000,0.146000,0.004000,0.246000,0.000000,0.000000,0.000000,0.000000
4,4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.592000,0.408000
5,5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
6,6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
7,7,escort,non_mandatory,0.000000,0.229722,0.000000,0.138687,0.000000,0.000000,0.000000,0.474643,0.156948,0.000000,0.000000
8,8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,9,eatout,non_mandatory,0.000000,0.786346,0.000000,0.213654,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
